In [ ]:
%load_ext autoreload
%autoreload 2

## Настройка окружения

In [ ]:
%pip install git+https://github.com/OPR-Project/OpenPlaceRecognition.git

In [ ]:
%pip install -r ../requirements.txt

In [ ]:
from pathlib import Path
import shutil
import cv2
import albumentations as A

import torchshow as ts
import torch
from torch import Tensor
from tqdm import tqdm
import pandas as pd

import opr
import numpy as np
import faiss

from opr.models.place_recognition import BoQModel, SequenceLateFusionModel
from opr.modules.temporal import TemporalAveragePooling
from opr.pipelines.place_recognition.sequential import SequencePlaceRecognitionPipeline

In [ ]:
print(f"PyTorch version: {torch.__version__}, cuda: {torch.cuda.is_available()}")
print(f"OpenPlaceRecognition version: {opr.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"FAISS version: {faiss.__version__}")

## Загрузка данных из S3

Используйте предоставленный скрипт: `python ../scripts/download_data.py --help` (рекомендуется запускать в отдельном терминале, а не в ноутбуке)

Или, например, перенесите код из скрипта и адаптируйте его под себя, если необходимо.

## Константы

In [ ]:
REPO_ROOT = Path.cwd().parent
print(f"Repository root dir: {REPO_ROOT}")

DATA_DIR = REPO_ROOT / "data"
assert DATA_DIR.exists(), f"Data directory {DATA_DIR} does not exist. Please run the download script."
print(f"Data dir: {DATA_DIR}")

SUBMISSIONS_DIR = REPO_ROOT / "submissions"
SUBMISSIONS_DIR.mkdir(exist_ok=True, parents=True)
print(f"Submissions dir: {SUBMISSIONS_DIR}")

## Чтение данных

Создадим DataReader для чтения данных с диска.

Ключевые моменты:
- `__getitem__` возвращает словарь с ключами:
  - `pose`: координаты в формате `[x, y]`
  - `image_front_cam`: изображение передней камеры (если указан аргумент `front_cam=True`)
  - `image_back_cam`: изображение задней камеры (если указан аргумент `back_cam=True`)
- `collate_fn` объединяет данные в батчи в нужном для OPR формате: словарь с ключами `poses`, `images_<camera_name>`, ...

In [ ]:
class ITLPTrackDataReader:
    def __init__(self, root: Path, image_transform: A.Compose, front_cam: bool = True, back_cam: bool = False):
        self._root = Path(root)
        self._front_cam_dir = self._root / "front_cam"
        self._back_cam_dir = self._root / "back_cam"

        self._track_df = pd.read_csv(self._root / "track.csv")
        self._image_transform = image_transform  # note that we use albumentations for image transformations
        self._front_cam = front_cam
        self._back_cam = back_cam
        if not self._front_cam and not self._back_cam:
            raise ValueError("At least one camera must be enabled: front_cam or back_cam.")

    def __len__(self) -> int:
        return len(self._track_df)

    def __getitem__(self, idx: int) -> dict[str, Tensor]:
        pose = self._track_df[["tx", "ty"]].iloc[idx].to_numpy()
        front_cam_path = self._front_cam_dir / f"{self._track_df['front_cam_ts'].iloc[idx]}.jpg"
        back_cam_path = self._back_cam_dir / f"{self._track_df['back_cam_ts'].iloc[idx]}.jpg"

        out_dict = {"pose": Tensor(pose)}

        if self._front_cam:
            front_cam_image = cv2.cvtColor(cv2.imread(str(front_cam_path)), cv2.COLOR_BGR2RGB)
            front_cam_image = self._image_transform(image=front_cam_image)["image"]  #
            out_dict["image_front_cam"] = front_cam_image
        if self._back_cam:
            back_cam_image = cv2.cvtColor(cv2.imread(str(back_cam_path)), cv2.COLOR_BGR2RGB)
            back_cam_image = self._image_transform(image=back_cam_image)["image"]
            out_dict["image_back_cam"] = back_cam_image

        return out_dict

    def collate_fn(self, batch: list[dict[str, Tensor]]) -> dict[str, Tensor]:
        collated_batch = {}
        for key in batch[0].keys():
            if key.startswith("image_"):
                collated_batch["images_" + key[6:]] = torch.stack([item[key] for item in batch])
            elif key == "pose":
                collated_batch["poses"] = torch.stack([item[key] for item in batch])
        return collated_batch

Для трансформов изображений используем библиотеку `albumentations`

In [ ]:
def setup_transforms(image_size: int = 322) -> A.Compose:  # 384 for ResNet50, 322 for DINOv2
    """Create image transformation pipeline."""
    return A.Compose(
        [
            A.CenterCrop(height=720, width=720),  # Crop to 720x720 for 1:1 aspect ratio
            A.Resize(height=image_size, width=image_size),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            A.pytorch.ToTensorV2(),
        ]
    )

In [ ]:
database_reader = ITLPTrackDataReader(
    root=DATA_DIR / "test" / "07_2023-10-04-day",
    image_transform=setup_transforms(image_size=322)  # Use 384 for ResNet50, 322 for DINOv2
)

In [ ]:
database_reader[0].keys()

In [ ]:
id_to_show = 1
ts.show(database_reader[id_to_show]["image_front_cam"])
print(f"Pose: {database_reader[id_to_show]['pose'].numpy()}")

## Инициализация модели


Сначала обычная модель для одиночных фреймов данных.

Здесь мы используем Bag-of-Queries (BoQ) с бэкбоном DINOv2.
О методе - https://github.com/amaralibey/Bag-of-Queries

In [ ]:
model = BoQModel(backbone_name="dinov2")

Модели в OPR ожидают определенный формат входных данных:
- `images_<camera_name>` - батч изображений для камеры `<camera_name>`


In [ ]:
sample_frame = database_reader[id_to_show]
sample_output = model(sample_frame)

**❗ Внимание:** `model.forward` ожидает на вход **батч** из словаря с ключами `images_<camera_name>` - размерность батча должна быть `(B, 3, H, W)`, где `B` - количество изображений в батче, `H` и `W` - высота и ширина изображений соответственно.

In [ ]:
batch_size = 1
database_dl = torch.utils.data.DataLoader(
    database_reader,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=database_reader.collate_fn,
    drop_last=False,
)

sample_batch = next(iter(database_dl))
print(f"Batch sample keys: {sample_batch.keys()}")
print(f"Batch sample shapes: {[v.shape for v in sample_batch.values()]}")

In [ ]:
sample_output = model(sample_batch)

print(f"Sample output keys: {sample_output.keys()}")
print(f"Sample output shapes: {[v.shape for v in sample_output.values()]}")
print(f"Sample descriptor shape: {sample_output['final_descriptor'].shape}")

## Подготовка БД

Для использования пайплайна инференса OPR необходимо подготовить базу данных в формате `faiss` индекса

In [ ]:
batch_size = 16
database_dl = torch.utils.data.DataLoader(
    database_reader,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=database_reader.collate_fn,
    drop_last=False,
)

In [ ]:
database_dir = DATA_DIR / "test" / "database"
database_dir.mkdir(parents=True, exist_ok=True)

model = model.to("cuda")
model.eval()

descriptors_list = []
with torch.no_grad():
    for batch in tqdm(database_dl):
        batch = {k: v.to("cuda") for k, v in batch.items()}
        descriptors = model(batch)["final_descriptor"]
        descriptors_list.append(descriptors)
descriptors = torch.cat(descriptors_list, dim=0)
print(f"Descriptors shape: {descriptors.shape}")

# Create L2 distance FAISS index for nearest neighbor search
faiss_index = faiss.IndexFlatL2(descriptors.shape[1])
faiss_index.add(descriptors.cpu().numpy())
faiss.write_index(
    faiss_index,
    str(database_dir / "index.faiss")
)

# Copy pose data as track.csv (required by PlaceRecognitionPipeline)
shutil.copy(DATA_DIR / "test" / "07_2023-10-04-day" / "track.csv", database_dir / "track.csv")

## Sequence-based baseline

В качестве бейзлайна для обработки последовательностей предлагается использовать алгоритм **Candidate Pool Fusion**:

![candidate_pool_fusion](../images/candidate_pool_fusion.jpg)

См. код в `opr.pipelines.place_recognition.sequential`

## Чтение query данных

Для удобства напишем небольшой Wrapper, который будет читать несколько фреймов из исходного DataReader и возвращать их в виде последовательности

In [ ]:
class TrackSeqWrapper:
    def __init__(self, track_data_reader: ITLPTrackDataReader, seq_len: int = 3):
        """Wrapper for ITLPTrackDataReader to provide sequences of specified length."""
        self.track_data_reader = track_data_reader
        self.seq_len = seq_len

    def __len__(self) -> int:
        return len(self.track_data_reader)

    def __getitem__(self, idx: int) -> list[dict[str, Tensor]]:
        """Get a sequence of frames up to the given index."""
        sequence = []
        for i in range(max(0, idx - self.seq_len + 1), idx + 1):
            sequence.append(self.track_data_reader[i])
        return sequence

In [ ]:
query_reader = ITLPTrackDataReader(
    root=DATA_DIR / "test" / "08_2023-10-11-night",
    image_transform=setup_transforms(image_size=322),
)

seq_data_reader = TrackSeqWrapper(
    track_data_reader=query_reader,
    seq_len=3,
)

## Инференс

См. реализации в коде библиотеки OPR:
- `opr.models.place_recognition.sequential.SequenceLateFusionModel`
- `opr.modules.temporal.TemporalAveragePooling`


In [ ]:
seq_model = SequenceLateFusionModel(
    model=model,
    temporal_fusion_module=TemporalAveragePooling()
)

pipe = SequencePlaceRecognitionPipeline(
    database_dir=DATA_DIR / "test" / "database",
    model=seq_model,
    use_candidate_pool_fusion=True,
)

In [ ]:
output_ids = []
for query_seq in tqdm(seq_data_reader):
    output = pipe.infer(query_seq)
    output_ids.append(output['idx'])

with open(SUBMISSIONS_DIR / "baseline.txt", "w") as f:
    for idx in output_ids:
        f.write(f"{idx}\n")

print(f"Submissions saved to {SUBMISSIONS_DIR / 'baseline.txt'}")